In [7]:
import os
import sklearn
import numpy as np
import pandas as pd
import tensorflow
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint ,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report, accuracy_score, log_loss
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

epoch_num = 50
p = input("Enter '0' if you want to train a new model, '1' to use a pretrained model and '2' to train a pretraind model")

In [8]:
path = '/kaggle/input/mushrooms-classification-common-genuss-images/mushrooms/Mushrooms/'
df = pd.DataFrame()
for imlabel in os.listdir(path):
    for image in os.listdir(path+imlabel):
        df = df.append({"path" : path+imlabel+"/"+image, "label" : imlabel}, ignore_index=True)

df.head()

## Splitting Data :
70%, 20%, 10% for training, validation and testing respecitvely

In [9]:
train_X, val_test_X, train_Y, val_test_Y = train_test_split(df['path'], df['label'], test_size=0.3, shuffle=True)
train_data = pd.concat({'path':train_X, 'label':train_Y}, axis=1)

val_X, test_X, val_Y, test_Y = train_test_split(val_test_X, val_test_Y, test_size=0.33, shuffle=True)

validation_data = pd.concat({'path':val_X, 'label':val_Y}, axis=1)
test_data = pd.concat({'path':test_X, 'label':test_Y}, axis=1)

In [10]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
vertical_flip = True)
train_data_generator = train_datagen.flow_from_dataframe(train_data,
                                                         x_col = 'path',
                                                         y_col = 'label',
                                                        batch_size=32,
                                                         color_mode="rgb",
                                                         target_size=(224, 224),
                                                        class_mode="categorical")
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_data_generator = validation_datagen.flow_from_dataframe(validation_data,
                                                                   x_col = 'path',
                                                                   y_col = 'label',
                                                                   batch_size=32,
                                                                   color_mode="rgb",
                                                                   target_size=(224, 224),
                                                                   class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_generator = test_datagen.flow_from_dataframe(test_data,
                                         x_col = 'path',
                                          y_col = 'label',
                                          batch_size=test_data['path'].shape[0],
                                          color_mode="rgb",
                                          target_size=(224, 224),
                                          class_mode="categorical")

#  DenseNet implementation

In [12]:
densenet1 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [13]:
img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet1(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(x)

model1 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model1.summary()

In [14]:
tensorflow.keras.utils.plot_model(model1)

In [15]:
early = EarlyStopping(monitor='val_accuracy', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.09,patience=2)

In [16]:
model1_path = '/kaggle/working/denseNet/Model.h5'
history1_path = '/kaggle/working/denseNet/model1_history.npy'

if ((not os.path.exists(model1_path)) or (not os.path.exists(history1_path))):
    history1 = model1.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
    history1 = history1.history
    model1.save(model1_path)
    np.save(history1_path,history1)

else:
    if (p=="0"):
        history1 = model1.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history1 = history1.history
        np.save(history1_path,history1)
        model1.save(model1_path)
    elif (p=="1"):
        model1 = tensorflow.keras.models.load_model(model1_path)
        history1=np.load(history1_path,allow_pickle='TRUE').item()
    elif (p=="2"):
        model1 = tensorflow.keras.models.load_model(model1_path)
        history1 = model1.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history1 = history1.history
        np.save(history1_path,history1)
        model1.save(model1_path)

In [17]:
accuracy1 = history1['accuracy']
val_accuracy1 = history1['val_accuracy']
loss1 = history1['loss']
val_loss1 = history1['val_loss']
precision1 = history1['precision']
val_precision1 = history1['val_precision']
recall1 = history1['recall']
val_recall1 = history1['val_recall']
f1_score1 = 2 * (np.array(precision1) * np.array(recall1)) / (np.array(precision1) + np.array(recall1))
val_f1_score1 = 2 * (np.array(val_precision1) * np.array(val_recall1)) / (np.array(val_precision1) + np.array(val_recall1))

import matplotlib.pyplot as plt

plt.plot(range(epoch_num), accuracy1, 'r', label='Training accuracy1')
plt.plot(range(epoch_num), val_accuracy1, 'b', label='Validation accuracy1')
plt.title('Training and validation accuracy1')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), loss1, 'r', label='Training loss1')
plt.plot(range(epoch_num), val_loss1, 'b', label='Validation loss1')
plt.title('Training and validation loss1')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), precision1, 'r', label='Training Precision1')
plt.plot(range(epoch_num), val_precision1, 'b', label='Validation Precision1')
plt.title('Training and validation Precision1')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), recall1, 'r', label='Training Recall1')
plt.plot(range(epoch_num), val_recall1, 'b', label='Validation Recall1')
plt.title('Training and validation Recall1')
plt.legend()

plt.figure()
plt.plot(range(epoch_num), f1_score1, 'r', label='Training Recall1')
plt.plot(range(epoch_num), val_f1_score1, 'b', label='Validation Recall1')
plt.title('Training and validation Recall1')
plt.legend()

### DenseNet Model performance on Test data.

In [18]:
for image, lab in test_data_generator:
    predict_y1 = model1.predict(image)
    real_lab1 = lab
    
    break;

print(f"Test Accuracy  :  {accuracy_score(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1))}")
print(f"Test Loss      :  {sklearn.metrics.zero_one_loss(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1))}")
print(f"Test Precision :  {precision_score(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1), average='macro')}")
print(f"Test Recall    :  {recall_score(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1), average='macro')}")
print(f"Test F1_Score  :  {f1_score(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1), average='macro')}")

## Confusion Matrix heatmap

In [19]:
cnf1 = tensorflow.math.confusion_matrix(np.argmax(real_lab1, axis=1), np.argmax(predict_y1, axis=1))
c_plot1 = sns.heatmap(cnf1, annot=True)
c_plot1.set(xlabel='true_value', ylabel='prediction')

In [20]:
def show_predicted_image1(i):
    plt.imshow(image[i])
    plt.title(os.listdir(path)[np.argmax(predict_y1[i])])
    plt.show()
    if (np.argmax(predict_y1[i])==np.argmax(real_lab1[i])):
        print("True prediction")
    else:
        print("Wrong value prediction")
        print(f"Correct label is : {os.listdir(path)[np.argmax(real_lab1[i])]}")
        
show_predicted_image1(4)

In [21]:
show_predicted_image1(7)

## Classification report for test datset

In [22]:
print(classification_report(np.argmax(lab, axis=1), np.argmax(predict_y1, axis=1)))

#  DenseNet + XceptionNet implementation

In [23]:
densenet2 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [24]:
xceptionNet2 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

In [25]:
img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet2(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

y = xceptionNet2(img_inputs)
y = tensorflow.keras.layers.GlobalAveragePooling2D()(y)
y = tensorflow.keras.layers.Dense(128, activation='relu')(y)

y = tensorflow.keras.layers.Concatenate()([x, y])
y = tensorflow.keras.layers.Dense(128, activation="relu")(y)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(y)

model2 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model2.summary()

In [26]:
tensorflow.keras.utils.plot_model(model2)

In [27]:
model2_path = '/kaggle/working/denseNet_XceptionNet/Model.h5'
history2_path = '/kaggle/working/denseNet_XceptionNet/model2_history.npy'

if ((not os.path.exists(model2_path)) or (not os.path.exists(history2_path))):
    history2 = model2.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
    history2 = history2.history
    model2.save(model2_path)
    np.save(history2_path,history2)

else:
    if (p=="0"):
        history2 = model2.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history2 = history2.history
        np.save(history2_path,history2)
        model2.save(model2_path)
    elif (p=="1"):
        model2 = tensorflow.keras.models.load_model(model2_path)
        history2=np.load(history2_path,allow_pickle='TRUE').item()
    elif (p=="2"):
        model2 = tensorflow.keras.models.load_model(model2_path)
        history2 = model2.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history2 = history2.history
        np.save(history2_path,history2)
        model2.save(model2_path)

In [28]:
accuracy2 = history2['accuracy']
val_accuracy2 = history2['val_accuracy']
loss2 = history2['loss']
val_loss2 = history2['val_loss']
precision2 = history2['precision']
val_precision2 = history2['val_precision']
recall2 = history2['recall']
val_recall2 = history2['val_recall']
f1_score2 = 2 * (np.array(precision2) * np.array(recall2)) / (np.array(precision2) + np.array(recall2))
val_f1_score2 = 2 * (np.array(val_precision2) * np.array(val_recall2)) / (np.array(val_precision2) + np.array(val_recall2))


plt.plot(range(epoch_num), accuracy2, 'g', label='Training accuracy2')
plt.plot(range(epoch_num), val_accuracy2, 'b', label='Validation accuracy2')
plt.title('Training and validation accuracy2')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), loss2, 'r', label='Training loss2')
plt.plot(range(epoch_num), val_loss2, 'b', label='Validation loss2')
plt.title('Training and validation loss2')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), precision2, 'r', label='Training Precision2')
plt.plot(range(epoch_num), val_precision2, 'b', label='Validation Precision2')
plt.title('Training and validation Precision2')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), recall2, 'r', label='Training Recall2')
plt.plot(range(epoch_num), val_recall2, 'b', label='Validation Recall2')
plt.title('Training and validation Recall2')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), f1_score2, 'r', label='Training F1_Score2')
plt.plot(range(epoch_num), val_f1_score2, 'b', label='Validation F1_Score2')
plt.title('Training and validation F1_Score2')
plt.legend()

## DenseNet+XceptionNet Performance on Test data.

In [29]:
for image, lab in test_data_generator:
    predict_y2 = model2.predict(image)
    real_lab2 = lab
    
    break;



print(f"Test Accuracy  :  {accuracy_score(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1))}")
print(f"Test Loss      :  {sklearn.metrics.zero_one_loss(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1))}")
print(f"Test Precision :  {precision_score(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1), average='macro')}")
print(f"Test Recall    :  {recall_score(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1), average='macro')}")
print(f"Test F1_Score  :  {f1_score(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1), average='macro')}")

## Confusion Matrix heatmap

In [30]:
cnf2 = tensorflow.math.confusion_matrix(np.argmax(real_lab2, axis=1), np.argmax(predict_y2, axis=1))
c_plot2 = sns.heatmap(cnf2, annot=True)
c_plot2.set(xlabel='prediction', ylabel='true_value')

In [31]:
def show_predicted_image2(i):
    plt.imshow(image[i])
    plt.title(os.listdir(path)[np.argmax(predict_y2[i])])
    plt.show()
    if (np.argmax(predict_y2[i])==np.argmax(real_lab2[i])):
        print("True prediction")
    else:
        print("Wrong value prediction")
        print(f"Correct label is : {os.listdir(path)[np.argmax(real_lab2[i])]}")
        
show_predicted_image2(4)

In [32]:
show_predicted_image2(7)

In [33]:
print(classification_report(np.argmax(lab, axis=1), np.argmax(predict_y2, axis=1)))

# DenseNet + XceptionNet + ResNet implementation

In [34]:
densenet3 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [35]:
xceptionNet3 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

In [36]:
resNet3 = tensorflow.keras.applications.resnet.ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(224, 224, 3)
)

In [37]:
img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet3(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

y = xceptionNet3(img_inputs)
y = tensorflow.keras.layers.GlobalAveragePooling2D()(y)
y = tensorflow.keras.layers.Dense(128, activation='relu')(y)

z = resNet3(img_inputs)
z = tensorflow.keras.layers.GlobalAveragePooling2D()(z)
z = tensorflow.keras.layers.Dense(128, activation='relu')(z)

z = tensorflow.keras.layers.Concatenate()([x, y, z])
z = tensorflow.keras.layers.Dense(128, activation="relu")(z)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(z)

model3 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model3.summary()

In [38]:
tensorflow.keras.utils.plot_model(model3)

In [39]:
model3_path = '/kaggle/working/denseNet_XceptionNet_ResNet/Model'
history3_path = '/kaggle/working/denseNet_XceptionNet_ResNet/model3_history.npy'

if ((not os.path.exists(model3_path)) or (not os.path.exists(history3_path))):
    history3 = model3.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
    history3 = history3.history
    model3.save(model3_path)
    np.save(history3_path,history3)

else:
    if (p=="0"):
        history3 = model3.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history3 = history3.history
        np.save(history3_path,history3)
        model3.save(model3_path)
    elif (p=="1"):
        model3 = tensorflow.keras.models.load_model(model3_path)
        history3=np.load(history3_path,allow_pickle='TRUE').item()
    elif (p=="2"):
        model3 = tensorflow.keras.models.load_model(model3_path)
        history3 = model3.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[early, reduce_lr])
        history3 = history3.history
        np.save(history3_path,history3)
        model3.save(model3_path)

In [40]:
accuracy3 = history3['accuracy']
val_accuracy3 = history3['val_accuracy']
loss3 = history3['loss']
val_loss3 = history3['val_loss']
precision3 = history3['precision']
val_precision3 = history3['val_precision']
recall3 = history3['recall']
val_recall3 = history3['val_recall']
f1_score3 = 2 * (np.array(precision3) * np.array(recall3)) / (np.array(precision3) + np.array(recall3))
val_f1_score3 = 2 * (np.array(val_precision3) * np.array(val_recall3)) / (np.array(val_precision3) + np.array(val_recall3))



plt.plot(range(epoch_num), accuracy3, 'g', label='Training accuracy3')
plt.plot(range(epoch_num), val_accuracy3, 'b', label='Validation accuracy3')
plt.title('Training and validation accuracy3')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), loss3, 'r', label='Training loss3')
plt.plot(range(epoch_num), val_loss3, 'b', label='Validation loss3')
plt.title('Training and validation loss3')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), precision3, 'r', label='Training Precision3')
plt.plot(range(epoch_num), val_precision3, 'b', label='Validation Precision3')
plt.title('Training and validation Precision3')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), recall3, 'r', label='Training Recall3')
plt.plot(range(epoch_num), val_recall3, 'b', label='Validation Recall3')
plt.title('Training and validation Recall3')
plt.legend()

plt.figure()

plt.plot(range(epoch_num), f1_score3, 'r', label='Training F1_Score3')
plt.plot(range(epoch_num), val_f1_score3, 'b', label='Validation F1_Score3')
plt.title('Training and validation F1_Score3')
plt.legend()

In [41]:
for image, lab in test_data_generator:
    predict_y3 = model3.predict(image)
    real_lab3 = lab
    
    break;


print(f"Test Accuracy  :  {accuracy_score(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1))}")
print(f"Test Loss      :   {sklearn.metrics.zero_one_loss(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1))}")
print(f"Test Precision :  {precision_score(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1), average='macro')}")
print(f"Test Recall    :  {recall_score(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1), average='macro')}")
print(f"Test F1_Score  :  {f1_score(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1), average='macro')}")

In [42]:
cnf3 = tensorflow.math.confusion_matrix(np.argmax(real_lab3, axis=1), np.argmax(predict_y3, axis=1))
c_plot3 = sns.heatmap(cnf3, annot=True)
c_plot3.set(xlabel='prediction', ylabel='true_value')

In [43]:
def show_predicted_image3(i):
    plt.imshow(image[i])
    plt.title(os.listdir(path)[np.argmax(predict_y3[i])])
    plt.show()
    if (np.argmax(predict_y3[i])==np.argmax(real_lab3[i])):
        print("True prediction")
    else:
        print("Wrong value prediction")
        print(f"Correct label is : {os.listdir(path)[np.argmax(real_lab3[i])]}")
        
show_predicted_image3(4)

In [44]:
show_predicted_image3(7)

In [45]:
print(classification_report(np.argmax(lab, axis=1), np.argmax(predict_y3, axis=1)))

# Plots for Comparison of Models

## 1. Train Accuracy plot.

In [46]:
plt.figure(figsize=(12, 9))
plt.plot(range(epoch_num), accuracy1, 'r', label='DenseNet Training accuracy')
plt.plot(range(epoch_num), accuracy2, 'b', label='DenseNet-XceptionNet Training accuracy')
plt.plot(range(epoch_num), accuracy3, 'g', label='DenseNet-XceptionNet-ResNet accuracy')
plt.plot(range(epoch_num), val_accuracy1, 'c', label='DenseNet Training accuracy')
plt.plot(range(epoch_num), val_accuracy2, 'm', label='DenseNet-XceptionNet Training accuracy')
plt.plot(range(epoch_num), val_accuracy3, 'y', label='DenseNet-XceptionNet-ResNet accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

## 2. Training - Validation Loss plot.

In [47]:
plt.figure(figsize=(12, 9))
plt.plot(range(epoch_num), loss1, 'r', label='DenseNet Trianing Loss')
plt.plot(range(epoch_num), loss2, 'b', label='DenseNet-XceptionNet Trianing Loss')
plt.plot(range(epoch_num), loss3, 'g', label='DenseNet-XceptionNet-ResNet Trianing Loss')
plt.plot(range(epoch_num), val_loss1, 'c', label='DenseNet Validation Loss')
plt.plot(range(epoch_num), val_loss2, 'm', label='DenseNet-XceptionNet Validation Loss')
plt.plot(range(epoch_num), val_loss3, 'y', label='DenseNet-XceptionNet-ResNet Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

## 3. Training - Validation Precision Plot

In [48]:
plt.figure(figsize=(12, 9))
plt.plot(range(epoch_num), precision1, 'r', label='DenseNet Training Precision')
plt.plot(range(epoch_num), precision2, 'b', label='DenseNet-XceptionNet Training Precision')
plt.plot(range(epoch_num), precision3, 'g', label='DenseNet-XceptionNet-ResNet Training Precision')
plt.plot(range(epoch_num), val_precision1, 'c', label='DenseNet Validation Precision')
plt.plot(range(epoch_num), val_precision2, 'm', label='DenseNet-XceptionNet Validation Precision')
plt.plot(range(epoch_num), val_precision3, 'y', label='DenseNet-XceptionNet-ResNet Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.show()

## 4. Training - Validation Recall Plot.

In [49]:
plt.figure(figsize=(12, 9))
plt.plot(range(epoch_num), recall1, 'r', label='DenseNet Training Recall')
plt.plot(range(epoch_num), recall2, 'b', label='DenseNet-XceptionNet Training Recall')
plt.plot(range(epoch_num), recall3, 'g', label='DenseNet-XceptionNet-ResNet Training Recall')
plt.plot(range(epoch_num), val_recall1, 'c', label='DenseNet Validation Recall')
plt.plot(range(epoch_num), val_recall2, 'm', label='DenseNet-XceptionNet Validation Recall')
plt.plot(range(epoch_num), val_recall3, 'y', label='DenseNet-XceptionNet-ResNet Validation Recall')
plt.xlabel("Epoch")
plt.ylabel("Recall")
plt.legend()
plt.show()

## 5. Training F1_Score plot

In [50]:
plt.figure(figsize=(12, 9))
plt.plot(range(epoch_num), f1_score1, 'r', label='DenseNet Training F1_Score')
plt.plot(range(epoch_num), f1_score2, 'b', label='DenseNet-XceptionNet Training F1_Score')
plt.plot(range(epoch_num), f1_score3, 'g', label='DenseNet-XceptionNet-ResNet Training F1_Score')
plt.plot(range(epoch_num), val_f1_score1, 'c', label='DenseNet Validation F1_Score')
plt.plot(range(epoch_num), val_f1_score2, 'm', label='DenseNet-XceptionNet Validation F1_Score')
plt.plot(range(epoch_num), val_f1_score3, 'y', label='DenseNet-XceptionNet-ResNet Validation F1_Score')
plt.xlabel('Epochs')
plt.ylabel('F1_Score')
plt.legend()
plt.show()

## All Models comparison for test dataset

In [51]:
for image, lab in test_data_generator:
    cmp = pd.DataFrame({'DenseNet' : model1.evaluate(image, lab), 'DenseNet+XceptionNet' : model2.evaluate(image, lab), 'DenseNet+XceptionNet+ResNet' : model3.evaluate(image, lab)}, index=['Loss', 'Accuracy', 'Precision', 'Recall'])
    break

In [52]:
cmp